In [28]:
import numpy as np
import pandas as pd

class Node:
    def __init__(self,key,value):
        self.left = None
        self.right = None
        self.parent = None
        self.value = value
        self.key = key
    
infile = open("GoldRush.txt")
inputdata = infile.read()
infile.close()
inputdata = inputdata.lower()
data = ""
for i in range(0,len(inputdata)):
    if inputdata[i]>='a' and inputdata[i]<='z':
        data+=(inputdata[i])
lst = []
for elem in data:
    lst.append(elem)
arr = pd.Series(lst)
freq = arr.value_counts(normalize=True, ascending=True)
freqdata = pd.DataFrame(freq)
freqdata.columns=['Frequency']
lst2 = []
while(freqdata.size!=1):
    num1 = freqdata.index[0]
    freq1 = freqdata.iloc[0]
    num2 = freqdata.index[1]
    freq2 = freqdata.iloc[1]
    freqnew = freq1 + freq2
    newkey = f"({num1},{num2})"
    left = Node(num1,freq1)
    right = Node(num2, freq2)
    parent = Node(newkey, freqnew)
    parent.left = left
    parent.right = right
    if left not in lst2:
        lst2.append(left)
    if right not in lst2:
        lst2.append(right)
    lst2.append(parent)
    freqdata.loc[newkey]=freqnew
    freqdata = freqdata.drop(index=[num1, num2])
    freqdata = freqdata.sort_values(by='Frequency',ascending=True)


root=lst2[-1]
mapping = pd.Series(freq)
for elem in mapping:
    encoding = ""
    for i in range(0,len(lst2)):
        if lst[i]==elem:
            curr = lst[i]
            print(curr)
   #
            

